# Deep Learning Fall 2023 Course Project - Zooming in on MLPs

### Imports

In [1]:
import os
import torch
import timm
import detectors

from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from ffcv.fields import BytesField, IntField, RGBImageField
from ffcv.writer import DatasetWriter
from transformers import AutoImageProcessor, AutoModelForImageClassification, ViTFeatureExtractor, ViTForImageClassification

from data_utils.data_stats import *
from data_utils.dataloader import get_loader
from data_utils.dataset_to_beton import get_dataset
from models.networks import get_model
from utils.metrics import topk_acc, real_acc, AverageMeter

from torchsummary import summary
import matplotlib.pyplot as plt

c:\Users\merci\miniconda3\envs\ffcv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Fetching data loader and model architecture

In [2]:
def get_data_and_model(dataset, model, data_path='/scratch/ffcv/'):
    """
    This function retrieves the data, model and feature extractor (if needed) based on the provided information.

    Parameters:
    dataset (str): The name of the dataset to retrieve (can be cifar10, cifar100 or imagenet).
    model (str): The name of the model to retrieve (can be mlp, cnn or vit; only mlp is supported for dataset imagenet).
    data_path (str): The path to the data.

    Returns (as a tuple):
    data_loader (DataLoader): The retrieved data loader.
    model (Model): The retrieved model.

    Raises:
    AssertionError: If the dataset or model is not supported.
    """

    assert dataset in ('cifar10', 'cifar100', 'imagenet'), f'dataset {dataset} is currently not supported by this function'
    assert model in ('mlp', 'cnn', 'vit'), f'model {model} is currently not supported by this function'

    num_classes = CLASS_DICT[dataset]
    eval_batch_size = 100

    if dataset == 'imagenet':
        data_resolution = 64
        assert model == 'mlp', f'imagenet dataset is only supported by mlp model'
    else:
        data_resolution = 32

    crop_resolution = data_resolution

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if device == 'cuda':
        torch.backends.cuda.matmul.allow_tf32 = True

    if model == 'mlp':
        architecture = 'B_12-Wi_1024'
        checkpoint = 'in21k_' + dataset
        model = get_model(architecture=architecture, resolution=64, num_classes=num_classes, checkpoint=checkpoint)

    if model == 'cnn':
        architecture = 'resnet18_' + dataset
        model = timm.create_model(architecture, pretrained=True)

    if model == 'vit':
        architecture = 'vit_small_patch16_224_' + dataset + '_v7.pth'
        model = torch.load(architecture)

    data_loader = get_loader(
        dataset,
        bs=eval_batch_size,
        mode="test",
        augment=False,
        dev=device,
        mixup=0.0,
        data_path=data_path,
        data_resolution=data_resolution,
        crop_resolution=crop_resolution,
    )
    model.cuda()
    return data_loader, model

In [3]:
class Reshape(torch.nn.Module): 
    def __init__(self, shape=224): 
        super(Reshape, self).__init__()
        self.shape = shape 
        
    def forward(self, x): 
        shape = self.shape
        x = transforms.functional.resize(x, size=(shape, shape))
        
        #if shape == 64, its an mlp
        if shape == 64:
            #x = torch.reshape(x, shape=(-1,))
            x = torch.reshape(x, shape=(x.shape[0],-1))
        return x

### Evaluating baseline model accuracy

In [ ]:
# Define a test function that evaluates test accuracy
@torch.no_grad()
def test(model, loader, is_mlp = False, is_vit = False):
    model.eval()
    total_acc, total_top5 = AverageMeter(), AverageMeter()

    for ims, targs in tqdm(loader, desc="Evaluation"):
        if is_mlp:
            model = torch.nn.Sequential(Reshape(64), model)
        if is_vit:
            model = torch.nn.Sequential(Reshape(224), model)
        
        preds = model(ims)    
        acc, top5 = topk_acc(preds, targs, k=5, avg=True)
        
        total_acc.update(acc, ims.shape[0])
        total_top5.update(top5, ims.shape[0])

    return (
        total_acc.get_avg(percentage=True),
        total_top5.get_avg(percentage=True),
    )

In [ ]:
data_loader, model = get_data_and_model(dataset='cifar10', model='mlp', data_path='/scratch/ffcv/')
test_acc, test_top5 = test(model, data_loader, is_mlp=True)

# Print all the stats
print("Test Accuracy        ", "{:.4f}".format(test_acc))
print("Top 5 Test Accuracy          ", "{:.4f}".format(test_top5))

### Evaluate adversarial accuracy

In [4]:
def denormalize(tensor, mean, std):
    """
    Denormalize a tensor.

    Parameters:
    tensor (torch.Tensor): The tensor to denormalize.
    mean (float or sequence): The mean used for normalization.
    std (float or sequence): The standard deviation used for normalization.

    Returns:
    torch.Tensor: The denormalized tensor.
    """
    return tensor*std[1]+mean[1]

def normalize(tensor, mean, std):
    """
    Normalize a tensor.

    Parameters:
    tensor (torch.Tensor): The tensor to normalize.
    mean (float or sequence): The mean used for normalization.
    std (float or sequence): The standard deviation used for normalization.

    Returns:
    torch.Tensor: The normalized tensor.
    """
    return (tensor-mean[1])/std[1]

def pgd(model, dataset, x_batch, label, eps, k, eps_step):
    """
    Performs the Projected Gradient Descent (PGD) for adversarial attacks.

    Parameters:
    model (torch.nn.Module): The model to attack.
    dataset (str): The name of the dataset used (can be cifar10, cifar100 or imagenet).
    x_batch (torch.Tensor): The input tensor.
    label (torch.Tensor): The true labels for the input tensor.
    eps (float): The maximum perturbation for PGD.
    k (int): The number of steps for PGD.
    eps_step (float): The step size for each iteration.

    Returns:
    torch.Tensor: The adversarially perturbed input tensor.
    """   
    mean, std = MEAN_DICT[dataset]/255, STD_DICT[dataset]/255

    x = x_batch.clone().detach_()
    x = denormalize(x, mean, std)
    x_adv = x + eps * (2*torch.rand_like(x) - 1)
    x_adv.clamp_(min=0., max=1.)
    
    for _ in range(int(k)):
        x_adv = normalize(x_adv, mean, std).detach_()
        x_adv.requires_grad_()
        model.zero_grad()
        loss = torch.nn.CrossEntropyLoss()(model(x_adv), label)
        loss.backward()
        perturbation = eps_step * x_adv.grad.sign()

        x_adv = denormalize(x_adv, mean, std)
        x_adv = x + (x_adv + perturbation - x).clamp_(min=-eps, max=eps)
        x_adv.clamp_(min=0, max=1)


    return normalize(x_adv.detach(), mean, std)

def fgsm_untargeted(model, dataset, x_batch, label, eps):
    """
    Performs the Fast Gradient Sign Method (FGSM) for untargeted adversarial attacks.

    Parameters:
    model (torch.nn.Module): The model to attack.
    dataset (str): The name of the dataset used (can be cifar10, cifar100 or imagenet).
    x_batch (torch.Tensor): The input tensor.
    label (torch.Tensor): The true labels for the input tensor.
    eps (float): The step size for the FGSM attack.

    Returns:
    torch.Tensor: The adversarially perturbed input tensor.
    """
    mean, std = MEAN_DICT[dataset]/255, STD_DICT[dataset]/255

    x = x_batch.clone().detach_()
    x.requires_grad_()
    model.zero_grad()
    loss = torch.nn.CrossEntropyLoss()(model(x), label)
    loss.backward()
    perturbation = eps * x.grad.sign()

    out = denormalize(x, mean, std) + perturbation
    out = out.clamp_(min=0, max=1)
        
    return normalize(out, mean, std)

In [5]:
def test_adversarial(model, dataset, loader, eps, mode, is_mlp = False, is_vit = False, modelname = 'MLP', datasetname = 'cifar10'):
    model.eval()
    total_adv_acc, total_adv_top5 = AverageMeter(), AverageMeter()
    batchnumber = 0
    if is_mlp:
        model = torch.nn.Sequential(Reshape(64), model)
    if is_vit:
        model = torch.nn.Sequential(Reshape(224), model)
    for ims, targs in tqdm(loader, desc="Evaluation"):

            
        if mode =="fgsm":
            adv_ims = fgsm_untargeted(model, dataset, ims, targs, eps)
            path = './adv_examples/' + modelname + '/' + datasetname + '/' + mode + '/' + str(eps) + 'Batch' + str(batchnumber)
            batchnumber += 1
            torch.save(adv_ims, path)
        if mode == "pgd":
            adv_ims = pgd(model, dataset, ims, targs, eps=eps, k=5, eps_step=eps/2)
            path = './adv_examples/' + modelname + '/' + datasetname + '/' + mode + '/' + str(eps) + 'Batch' + str(batchnumber)
            batchnumber += 1
            torch.save(adv_ims, path)
        adv_preds = model(adv_ims)
        adv_acc, adv_top5 = topk_acc(adv_preds, targs, k=5, avg=True)
        total_adv_acc.update(adv_acc, ims.shape[0])
        total_adv_top5.update(adv_top5, ims.shape[0])

    return (
        total_adv_acc.get_avg(percentage=True),
        total_adv_top5.get_avg(percentage=True),
    )

In [7]:
#USE BATCH SIZE 100

#already done: MLP(everything),cnn everytthing, vittt fgp,pgd on cif10 pgd on cifar100..

eps_range = 0.025
steps = 12

torch.cuda.empty_cache()
for modelname in ['vit']:
    is_mlp = modelname == 'mlp'
    is_vit = modelname == 'vit'
    for datasetname in ['cifar100']:
        for methodname in [ 'pgd']:
            adv_acc = []
            adv_top5 = []
            print('Now starting: ' + methodname +' on ' + modelname + ' with ' + datasetname)
            data_loader, model = get_data_and_model(dataset=datasetname, model=modelname, data_path='./beton/')
            #all_eps = np.arange(0,0.26,0.0125)
            all_eps = np.arange(0,eps_range,eps_range / steps)
            for eps in tqdm(all_eps, desc="Evaluating"):
                test_adv_acc, test_adv_top5 = test_adversarial(model, datasetname, data_loader, eps, methodname, is_vit = is_vit, is_mlp= is_mlp, modelname= modelname, datasetname= datasetname)
                adv_acc.append(test_adv_acc)
                adv_top5.append(test_adv_top5)

            name = methodname + '_' + modelname + '_' + datasetname + '_zoomedin_'
            np.save('accuracy_' + name, adv_acc)
            np.save('top5_' + name, adv_top5)

Now starting: pgd on vit with cifar100
Loading ./beton/cifar100\ffcv\test\test_32.beton


Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]c:\Users\merci\miniconda3\envs\ffcv\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Evaluating: 100%|██████████| 12/12 [45:34<00:00, 227.84s/it]


In [8]:
#plot adversarial accuracies


#all_eps = np.arange(0,0.26,0.0125)
eps_range = 0.025
steps = 12
all_eps = np.arange(0,eps_range,eps_range / steps)
for modelname in ['cnn', 'vit', 'mlp']:
    for datasetname in ['cifar10','cifar100']:
        for mode in ['pgd']:
            name = mode + '_' + modelname + '_' + datasetname + '_zoomedin_'
            accuracy = np.load('accuracy_' +name+ '.npy')
            accuracytop5 = np.load('top5_' +name+ '.npy')
            
        

            #plot accuracies
            plt.plot(all_eps, accuracy)
            plt.title('accuracy_' +name)
            plt.savefig('./plots/'+ 'accuracy_'+ name  +  '.jpg')
            plt.clf()

            #plot top5 accuracies
            plt.plot(all_eps, accuracy)
            plt.title('top5_' +name)
            plt.savefig('./plots/' +'top5_'+ name + '.jpg')
            plt.clf()


<Figure size 640x480 with 0 Axes>

In [ ]:
#plot adversarial examples

for modelname in [ 'mlp']:
    for datasetname in ['cifar10','cifar100']:
        data_loader, model = get_data_and_model(dataset=datasetname, model=modelname, data_path='./beton/')
        for mode in ['fgsm','pgd']:
            for eps in tqdm(all_eps, desc="Evaluating"):
                batchnumber = 0
                for x,y in data_loader:
                   
                    
                    #get adversarial batch
                    path = './adv_examples/' + modelname + '/' + datasetname + '/' + mode + '/' + str(eps) + 'Batch' + str(batchnumber)
                    batchnumber += 1
                    adv_batch = torch.load(path)
                    adv_im = adv_batch[0].cpu().permute(1, 2, 0).detach().numpy()
                    im = x[0].cpu().permute(1, 2, 0).detach().numpy()

                    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
                    #plot standart image and adv 
                    axes[0].imshow(im)
                    axes[0].set_title(modelname +  datasetname +  mode +  str(eps) + 'Batch' + str(batchnumber))
                    plt.subplot(1, 2, 1)
                    axes[1].imshow(adv_im)
                    axes[1].set_title('aversarial' + modelname +  datasetname +  mode +  str(eps) + 'Batch' + str(batchnumber)) 
                    plt.tight_layout()
                    plt.clf()
                    break             


In [8]:
def test_accuracy_advexamples(model, datamodel, datasetname, eps, mode, loader):
    #testmodel: Model to be tested
    #datamodel: Model with which the adv examples were generated
    #datasetname: from which dataset to take the adv examples
    #eps & mode: params for adversarial generation

    

    total_adv_acc, total_adv_top5 = AverageMeter(), AverageMeter()
    batchnumber = 0


    for ims, targs in tqdm(loader, desc="Evaluation"):

        if mode =="fgsm":
            path = './adv_examples/' + datamodel + '/' + datasetname + '/' + mode + '/' + str(eps) + 'Batch' + str(batchnumber)
            adv_ims = torch.load(path)
            batchnumber += 1
            
        if mode == "pgd":
            path = './adv_examples/' + datamodel + '/' + datasetname + '/' + mode + '/' + str(eps) + 'Batch' + str(batchnumber)
            adv_ims = torch.load(path)
            batchnumber += 1
            
        adv_preds = model(adv_ims)
        adv_acc, adv_top5 = topk_acc(adv_preds, targs, k=5, avg=True)
        total_adv_acc.update(adv_acc, ims.shape[0])
        total_adv_top5.update(adv_top5, ims.shape[0])

    return (
        total_adv_acc.get_avg(percentage=True),
        total_adv_top5.get_avg(percentage=True),
    )

In [11]:
#Get adversarial transversability examples
top_acc = []
top5_acc = []
top_comparison = []
for testmodel in ['cnn']:
    for datasetname in ['cifar10']:
        loader, model = get_data_and_model(dataset=datasetname, model=testmodel, data_path='./beton/')
        if testmodel == 'mlp':
            model = torch.nn.Sequential(Reshape(64), model)
        if testmodel == 'vit':
            model = torch.nn.Sequential(Reshape(224), model)
        model.cuda()
        model.eval()
        for datamodel in ['mlp']:
            for eps in np.arange(0,0.26,0.0125):
                for mode in ['fgsm']:
                    total_adv_acc, total_adv_top5 = test_accuracy_advexamples(model, datamodel, datasetname,eps, mode, loader)
                    #total_adv_acc_comparison, total_adv_top5_comparison = test_accuracy_advexamples(model, testmodel, datasetname,eps, mode, loader)

                    top_acc.append(total_adv_acc)
                    top5_acc.append(total_adv_top5)
                    #top_comparison.append(total_adv_acc_comparison)

Loading ./beton/cifar10\ffcv\val\val_32.beton


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]Exception ignored in: <finalize object at 0x223e4331920; dead>
Traceback (most recent call last):
  File "c:\Users\merci\miniconda3\envs\ffcv\lib\weakref.py", line 591, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "c:\Users\merci\miniconda3\envs\ffcv\lib\site-packages\numba\core\dispatcher.py", line 312, in finalizer
    for cres in overloads.values():
KeyError: (Array(uint8, 1, 'C', True, aligned=True), Array(uint8, 1, 'C', True, aligned=True), uint32, uint32, uint32, uint32, Literal[int](0), Literal[int](0), Literal[int](1), Literal[int](1), Literal[bool](False), Literal[bool](False))
Evaluation: 100%|██████████| 100/100 [00:27<00:00,  3.68it/s]


In [10]:
print(top_acc)
print(top_comparison)

[93.78999847173691, 88.03999811410904, 80.89999812841415, 73.23999810218811, 65.96999871730804, 58.54999881982803, 51.52999863028526, 44.42999878525734, 38.14999911189079, 32.68999923765659, 28.43999920785427, 24.729999244213104, 21.479999467730522, 19.02999947220087, 17.009999625384808, 15.629999622702599, 14.679999627172947, 13.759999677538872, 13.489999629557133, 12.92999967560172, 12.559999715536833]
[93.78999847173691, 23.439999356865883, 21.039999529719353, 22.97999933362007, 25.189999282360077, 26.78999924659729, 27.349999263882637, 26.539999306201935, 25.6199993789196, 24.049999356269836, 22.019999355077744, 20.229999475181103, 17.90999949723482, 16.21999954432249, 14.859999641776085, 13.909999668598175, 13.09999967738986, 12.459999728947878, 12.14999964274466, 12.13999966904521, 12.019999668002129]


## Marcel's part
Run cells 1, 2, 5 & 6 before this and change data_path below

In [ ]:
for modelname in ['vit', 'cnn']:
    for datasetname in ['cifar10', 'cifar100']:
        is_mlp = modelname == 'mlp'

        adv_acc = []
        adv_top5 = []
        data_loader, model = get_data_and_model(dataset=datasetname, model=modelname, data_path='./beton/')
        all_eps = np.arange(0,0.26,0.0125)

        for eps in tqdm(all_eps, desc="Evaluating"):
            test_adv_acc, test_adv_top5 = test_adversarial(model, datasetname, data_loader, eps, 'pgd', is_mlp = is_mlp, modelname= modelname, datasetname= datasetname)

            adv_acc.append(test_adv_acc)
            adv_top5.append(test_adv_top5)

        acc_fname = 'pgd_acc_' + model + '_' + datasetname
        top5_fname = 'pgd_top5_' + model + '_' + datasetname
        
        np.save(acc_fname, np.array(adv_acc))
        np.save(top5_fname, np.array(adv_top5))